## Instala y arranca Cassandra

#### descarga e instala cassandra en `/opt` DESDE UN TERMINAL
    > wget -N http://apache.mirrors.hoobly.com/cassandra/2.1.8/apache-cassandra-2.1.8-bin.tar.gz
    > sudo tar zxv -C /opt  -f apache-cassandra-2.1.8-bin.tar.gz > /dev/null
    
#### apaga algunos servicios para liberar memoria

    > sudo service ambari-agent stop
    > sudo service ambari-server stop
    
#### arranca el servidor de cassandra EN EL TERMINAL

    > sudo /opt/apache-cassandra-2.1.8/bin/cassandra -f

## Tablas y queries

In [26]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh

DROP KEYSPACE myspace;
CREATE KEYSPACE myspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};

In [33]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh

use myspace;
drop table users;
drop table collections_example;
create table users (username text primary key, password text);
CREATE TABLE collections_example (    id int PRIMARY KEY,    set_example set<text>,    list_example list<text>,    map_example map<int,text> );

In [34]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh

describe keyspace myspace;
describe columnfamily collections_example;


CREATE KEYSPACE myspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}  AND durable_writes = true;

CREATE TABLE myspace.collections_example (
    id int PRIMARY KEY,
    list_example list<text>,
    map_example map<int, text>,
    set_example set<text>
) WITH bloom_filter_fp_chance = 0.01
    AND caching = '{"keys":"ALL", "rows_per_partition":"NONE"}'
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy'}
    AND compression = {'sstable_compression': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND dclocal_read_repair_chance = 0.1
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair_chance = 0.0
    AND speculative_retry = '99.0PERCENTILE';

CREATE TABLE myspace.users (
    username text PRIMARY KEY,
    password text
) WITH bloom_filter_fp_chance

<stdin>:4:Keyspace None not found.


In [36]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh

use myspace;

insert into collections_example (id, set_example) values (1, {'1-one', '2-two'});
UPDATE collections_example SET set_example = set_example + {'3-three'} WHERE id=1;
UPDATE collections_example SET set_example = set_example - {'3-three'} WHERE id = 1;

INSERT INTO collections_example (id, list_example) VALUES (1,['1-one','2-two']) ;
update collections_example set list_example = list_example + [ '3-three' ] where id = 1;
update collections_example set list_example = list_example + [ '0-zero' ] where id = 1;

insert into collections_example (id, map_example) values (1, { 1: 'one', 2: 'two' } );
update collections_example set map_example[3]='tres' where id=1;

In [37]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh

use myspace;
select * from collections_example;


 id | list_example                            | map_example                     | set_example
----+-----------------------------------------+---------------------------------+--------------------
  1 | ['1-one', '2-two', '3-three', '0-zero'] | {1: 'one', 2: 'two', 3: 'tres'} | {'1-one', '2-two'}

(1 rows)


In [38]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh
use myspace;

delete map_example[2] from collections_example where id=1;
select * from collections_example;


 id | list_example                            | map_example           | set_example
----+-----------------------------------------+-----------------------+--------------------
  1 | ['1-one', '2-two', '3-three', '0-zero'] | {1: 'one', 3: 'tres'} | {'1-one', '2-two'}

(1 rows)


In [39]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh
use myspace;

alter table collections_example add name text;
select * from collections_example;


 id | list_example                            | map_example           | name | set_example
----+-----------------------------------------+-----------------------+------+--------------------
  1 | ['1-one', '2-two', '3-three', '0-zero'] | {1: 'one', 3: 'tres'} | null | {'1-one', '2-two'}

(1 rows)


In [41]:
%%script /opt/apache-cassandra-2.1.8/bin/cqlsh
use myspace;

consistency
consistency all

Current consistency level is ONE.
Consistency level set to ALL.
